In [1]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [2]:
raw_train = pd.read_csv('train.csv')
raw_test = pd.read_csv('test.csv')
raw_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
train_df = raw_train.copy()
test_df = raw_test.copy()

In [4]:
test_df.info() # only for prediction

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            362 non-null    object 
 1   Gender             351 non-null    object 
 2   Married            362 non-null    object 
 3   Dependents         353 non-null    object 
 4   Education          362 non-null    object 
 5   Self_Employed      339 non-null    object 
 6   ApplicantIncome    362 non-null    int64  
 7   CoapplicantIncome  362 non-null    int64  
 8   LoanAmount         362 non-null    int64  
 9   Loan_Amount_Term   356 non-null    float64
 10  Credit_History     333 non-null    float64
 11  Property_Area      362 non-null    object 
dtypes: float64(2), int64(3), object(7)
memory usage: 34.1+ KB


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [6]:
train_y = train_df['Loan_Status'].copy()

In [7]:
train_df.drop(columns = ['Loan_Status'],inplace=True)

In [8]:
# Dropping the unncessary columns
train_df.drop(columns='Loan_ID',inplace=True)
test_df.drop(columns='Loan_ID',inplace=True)

In [9]:
train_df.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area'],
      dtype='object')

In [10]:
# Duplicates --> no duplicates
train_df[train_df.duplicated()]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area


In [11]:
test_df[test_df.duplicated()]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
192,Male,No,0,Graduate,Yes,5833,0,116,360.0,1.0,Urban


In [12]:
test_df.drop_duplicates(inplace=True)

In [13]:
# Missing Value analysis
train_df.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
dtype: int64

In [14]:
train_df.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area'],
      dtype='object')

In [15]:
# Numeric --> mean
# Categorical --> mode

num_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term']

cat_cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
           'Credit_History', 'Property_Area']

In [16]:
cat_imputer = SimpleImputer(strategy="most_frequent")
cat_imputer.fit(train_df[cat_cols])

train_df[cat_cols] = cat_imputer.transform(train_df[cat_cols])
test_df[cat_cols] = cat_imputer.transform(test_df[cat_cols]) 

In [17]:
num_imputer = SimpleImputer(strategy="mean")
num_imputer.fit(train_df[num_cols])

train_df[num_cols] = num_imputer.transform(train_df[num_cols])
test_df[num_cols] = num_imputer.transform(test_df[num_cols]) 

In [18]:
# Missing Value analysis
train_df.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [19]:
# preprocessing as per the domain knowledge
train_df['ApplicantIncome'] = train_df['ApplicantIncome'] + train_df['CoapplicantIncome']
test_df['ApplicantIncome'] = test_df['ApplicantIncome'] + test_df['CoapplicantIncome']

# drop the co-applicant income columne
train_df.drop(columns='CoapplicantIncome',inplace=True)
test_df.drop(columns='CoapplicantIncome',inplace=True)

In [20]:
# Application of Label Encoder
train_df.nunique()

Gender                2
Married               2
Dependents            4
Education             2
Self_Employed         2
ApplicantIncome     554
LoanAmount          204
Loan_Amount_Term     11
Credit_History        2
Property_Area         3
dtype: int64

In [21]:
train_df.Dependents.unique() 

array(['0', '1', '2', '3+'], dtype=object)

In [22]:
for col in cat_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])

In [23]:
num_cols.remove('CoapplicantIncome')

In [24]:
#scaling
minmax = MinMaxScaler()
train_df = minmax.fit_transform(train_df)
test_df = minmax.transform(test_df)

In [25]:
# Building the Model
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(train_df,train_y,test_size=0.3, random_state=0)

In [26]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(X_train,y_train)

LogisticRegression()

In [28]:
y_pred_test = log.predict(X_test)

In [29]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test,y_pred_test)
print(f"Accuracy is {acc}")

Accuracy is 0.827027027027027


# Serialization and Deserialization

In [30]:
# serialization & deserialization
import joblib
joblib.dump(log,"my_trained_model_v1.pkl")

['my_trained_model_v1.pkl']

In [31]:
#deserialization
final_model = joblib.load("my_trained_model_v1.pkl")

# Package and Modules

# Create Custom Data Transformers

In [33]:
# Key thing --> Inherit - BaseEstimator, TransformerMixin
# implement fit and transform
# accept input with __init__ method

from sklearn.base import BaseEstimator,TransformerMixin

class DemoTransformer(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self,X,y=None):
        return self

    def transform(self,X):
        return X

In [34]:
# Numerical Imputation - mean
from sklearn.base import BaseEstimator,TransformerMixin

class MeanImputer(BaseEstimator,TransformerMixin):
    def __init__(self,variables=None):
        self.variables = variables
    
    def fit(self,X,y=None):
        self.mean_dict = {}
        for col in self.variables:
            self.mean_dict[col] = X[col].mean()
        return self
    
    def transform(self,X):
        X = X.copy()
        for col in self.variables:
            X[col].fillna(self.mean_dict[col],inplace=True)
        return X

In [35]:
np.random.seed(0)
df = pd.DataFrame(np.random.randint(0,100,(10,2)),columns=["A",'B'])
df.iloc[1,0] = np.nan
df.iloc[2,1] = np.nan
df.iloc[3,1] = np.nan
df.iloc[4,0] = np.nan
df

,A,B
0,44.0,47.0
1,NaN,67.0
2,67.0,NaN
3,83.0,NaN
4,NaN,87.0
5,70.0,88.0
6,88.0,12.0
7,58.0,65.0
8,39.0,87.0
9,46.0,88.0


In [36]:
mean_imputer = MeanImputer(variables=["A",'B'])

In [37]:
mean_imputer.fit(df)

MeanImputer(variables=['A', 'B'])

In [38]:
mean_imputer.mean_dict

{'A': 61.875, 'B': 67.625}

In [39]:
mean_imputer.transform(df)

,A,B
0,44.000,47.000
1,61.875,67.000
2,67.000,67.625
3,83.000,67.625
4,61.875,87.000
5,70.000,88.000
6,88.000,12.000
7,58.000,65.000
8,39.000,87.000
9,46.000,88.000


In [40]:
import numpy
numpy.__version__

'1.24.3'

In [41]:
import scipy
scipy.__version__

'1.12.0'